# Analiza objaśniająca zbioru Titanic

## Wstęp

RMS Titanic, brytyjski statek pasażerski kursujący pamiędzy USA i Europą. Z nocy 14 na 15 kwietnia
1912 roku, podczas pierwszego rejsu, zderzył się z górą lodową i zatonął. Tragiczna katastrofa, w wyniku
której zginęło ponad 1500 osób, wpłynęła w znacznym stopniu na nowelizację i poprawę zasad bezpieczeństwa
morskiego. Ponadto w wyniku opublikowania raportu Amerykańskiej Komisji śledczej, doszło do przesunięcia
szlaku żeglugowego na północnym Atlantyku o 60 mil na południe.

### Cel opracowania

Celem opracowania nie jest sprawdzenie jakie działania mogły zapobiec katastrofie Titanica, czy ograniczyć zakres zebranego przez nią rzniwa. Jest nim natomiast odpowiedzenie na pytanie, kto miał największe, a kto najmniejsze szanse na przetrwanie. 

## Analiza zbioru

Importujemy zbiór danych, w którym wszystkie puste pola przyjmą wartość NA.

In [ ]:
titanic <- read.csv("Titanic.csv", na.strings=c("","NA"))

<br>Wyświetlamy statystykę podsumowującą w celu zapoznania się ze zbiorem.

In [ ]:
summary(titanic)

<br>

Zbiór składa się z jedenastu atrybutów opisujących pasażerów statku:
<ul>
    <li><b>PassengerId</b> - identyfikator</li>
    <li><b>Pclass</b> - klasa podróżnicza, przyjmująca wartości: <i>1, 2, 3</i></li>
    <li><b>Name</b> - imie</li>
    <li><b>Sex</b> - płeć, przyjmująca wartości: <i>male, female</i></li>
    <li><b>Age</b> - wiek</li>
    <li><b>SibSp</b> - ilość rodzeństwa/ małżonków na statku</li>
    <li><b>Parch</b> - ilość rodziców/ dzieci na statku</li>
    <li><b>Ticket</b> - kod biletu</li>
    <li><b>Fare</b> - opłata za przeprawę</li>
    <li><b>Cabin</b> - numer kabiny</li>
    <li><b>Embarked</b> - port startowy, przyjmujący wartości: <i>S, C, Q</i></li>
</ul>
Oraz atrybutu decyzyjnego:
<ul>
    <li><b>Survived</b> - przetrwanie, przyjmujące wartości: <i>0, 1</i></li>
</ul>

<br>Sprawdzimy jak prezentują się brakujące wartości w zbiorze.

In [ ]:
library(Amelia)
missmap(titanic, col=c("black","grey"), main = "Mapa brakujących wartości")

Kolumna <b>Cabin</b> zawiera bardzo dużo pustych wartości, co oznacza, że nie będzie rzetelnym wyznacznikiem szans przeżycia na Titanicu. Zostanie usnięta razem z <b>PassengerId, Name i Ticket</b> które równie niewiele wniosą w dalszą analizę.

In [ ]:
library(dplyr)
titanic <- select(titanic, Survived, Age, Sex, SibSp, Parch, Pclass, Fare, Embarked)

<br>Usuwamy wsztstkie wiersze z brakującymi wartościami atrybutu <b>Age</b>.

In [ ]:
titanic <- na.omit(titanic)

<br>Przed przejściem do wizualizacji, przyglądamy się strukturze danych.

In [ ]:
str(titanic)

<br>Kolumny <b>Survived, Sex i Pclass</b> zostaną potraktowane jako zmienne kategoryzujące.

In [ ]:
titanic$Survived <- factor(titanic$Survived)
titanic$Sex <- factor(titanic$Sex)
titanic$Pclass <- factor(titanic$Pclass, order=TRUE, levels = c(3, 2, 1))

### Wizualizacja danych

W pierwszej kolejności sprawdzimy korelacje między atrybutami numerycznymi.

In [ ]:
library(GGally)
ggcorr(titanic,nbreaks = 6,label = TRUE,label_size = 4,color = "black")

Najsilniejsza korelacja występuje między atrybutem <b>SibSp i Parch</b>, co sugeruje, że rodziny podróżowały razem.

In [ ]:
library(ggplot2)
ggplot(titanic, aes(x = Survived, fill=Sex)) + 
geom_bar(position = position_dodge()) +
geom_text(stat='count', aes(label=stat(count)),
position = position_dodge(width=1), vjust=-0.3) +
xlab("Przetrwanie") + ylab("Liczba pasażerów") +
labs(title = "Przetrwanie na Titanicu wg. płci",
caption = "1 - przetrwanie, 0 - śmierć ") +
theme_classic()


O wiele większe prawdopodobieństwo na przetrwanie katastrofy Titanica miały kobiety niż mężczyźni.

In [ ]:
ggplot(titanic, aes(x = Survived, fill=Pclass)) +
geom_bar(position = position_dodge()) +
geom_text(stat='count', aes(label=stat(count)),
position = position_dodge(width=1), vjust=-0.3) +
xlab("Przetrwanie") + ylab("Liczba pasażerów") +
labs(title = "Przetrwanie na Titanicu wg. klasy",
caption = "1 - przetrwanie, 0 - śmierć ") +
theme_classic()


Największe prawdopodobieństwo przetrwania katastrofy miały osoby podróżujące pierwszą klasą.

In [ ]:
ggplot(titanic, aes(x = Age)) +
geom_density(fill='dodgerblue3') +
xlab("Wiek") + ylab("Gęstość") +
labs(title = "Rozkład wieku pasażerów na Titanicu") +
theme_minimal()

Większością pasażerów Titanica były osoby dorosłe w przedziale wiekowym 20-40 lat.

In [ ]:
titanic$Discretized.age = cut(titanic$Age, c(0,10,20,30,40,50,60,70,80,100))
ggplot(titanic, aes(x = Discretized.age, fill=Survived)) +
geom_bar(position = position_dodge()) +
geom_text(stat='count', aes(label=stat(count)),
position = position_dodge(width=1), vjust=-0.3)+
xlab("Przedział wiekowy") + ylab("Liczba pasażerów") +
labs(title = "Zawartość grup wiekowych na Titanicu") +
theme_classic()

Najczęściej ratowaną grupą wiekową na Titanicu były dzieci przed dziesiątym rokiem życia.

## Wnioski

- Katastrofę przetrwało 288 z 712 osób, prawie dwóch na trzech pasażerów nigdy nie dotarło do Nowego Yorku. 
- Sześć na siedem ofiar stanowili mężczyźni.
- Spośród 288 ocalonych połową były kobiety. 
- Największą śmiertelnością cechowała się trzecia klasa - trzy na cztery osoby, za nią klasa druga - jedna na dwie, i pierwsza - jedna na trzy.
- Większością pasażerów były osoby miedzy 20 a 40 rokiem życia. 
- <b>Najwyższą szansę na przetrwanie miały dzieci przed 10 rokiem życia - prawie 60%, najniższą dorośli między 20 a 30 - okolice 35%.</b>